In [76]:
class Tree(object):#括号里是继承的父类#
    #树#
    class Position(object):
        def element(self):
            raise NotImplementedError('must implemented by subclass')
            
        def __eq__(self,other):#==被__eq__重载#
            raise NotImplementedError('must implemented by subclass')
            
        def __neq__(self,other):#~=被__neq__重载#
            return not (self==other)
    
    def root(self):
        raise NotImplementedError('must implemented by subclass')
        
    def parent(self,p):#找父节点#
        raise NotImplementedError('must implemented by subclass')
    
    def num_children(self,p):#找孩子节点数量#
        raise NotImplementedError('must implemented by subclass')
    
    def children(self,p):#找孩子节点#
        raise NotImplementedError('must implemented by subclass')
        
    def positions(self):
        raise NotImplementedError('must implemented by subclass')
    
    def __len__(self):#节点总数#
        raise NotImplementedError('must implemented by subclass')
        
    def is_root(self,p):
        return p == self.root()
    
    def is_leaf(self,p):
        return self.num_children(p) == 0
    
    def is_empty(self):
        return len(self) == 0
    
    def depth(self,p):
        if self.is_root():
            return 0
        else:
            return self.depth(self.parent(p)) + 1
    
    def _height(self,p):
        if self.is_leaf():
            return 0
        else:
            return max([self._height(c) for c in self.children(p)]) + 1
            #max方法对list进行操作，应该用列表生成式写递归#
    
    def height(self,p=None):
        if p==None:
            p=self.root()
        return self._height(p)
    
    def height1(self):#树的高度，时间复杂度O(n^2)#
        return max([self.depth(p) for p in self.positions() if self.is_leaf(p)])
    
    def _preorder(self):
        #前序遍历#
        if not self.is_empty():
            for p in self._subtree_preoder(self.root()):
                yield p
    def _subtree_preoder(self,p):
        #对子树的前序遍历#
        """Generate a preorder iteration of positions in subtree rooted at p."""
        yield p
        for c in self.children(p):
            for other in self._subtree_preoder(c):
                yield other
                
    def _postorder(self):
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p    
    def _subtree_postorder(self,p):
        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other
        yield p
        
    def breadthfirst(self):
        #广度优先遍历#
        #没有用到递归，任何队列ADT都可以#
        if not self.is_empty():
            Q=LinkedQueue()
            Q.enqueue(self.root())
            while not Q.is_empty():
                p=Q.dequeue()
                yield p
                for c in self.children(p):
                    Q.enqueue(c)

In [77]:
class LinkedQueue(object):
    class _Node(object):
        def __init__(self,element,next):
            self._element=element
            self._next=next
            
    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0
    def __len__(self):
        return self._size
    def is_empty(self):
        return self._size == 0
    def first(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._head._element
    def dequeue(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        result = self._head._element
        self._head=self._head._next
        self._size -=1
        if self.is_empty():
            self._tail = None
        return result
    def enqueue(self,e):
        node = self._Node(e,None)
        if self.is_empty():
            self._head= node
        else:    
            self._tail._next=node
        self._tail=node
        self._size+=1

In [78]:
class BinaryTree(Tree):
    #二叉树#
    #抽象基类没有init方法#
    def left(self,p):
        raise NotImplementedError('must implemented by subclass')
    
    def right(self,p):
        raise NotImplementedError('must implemented by subclass')
    
    def sibling(self,p):#返回兄弟节点的位置#
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p ==self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
            
    def children(self,p):
        if self.left(p) is not None: 
            yield self.left(p) 
        if self.right(p) is not None: 
            yield self.right(p)
            
    def _inorder(self):
        #中序遍历#
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p
    
    def _subtree_inorder(self,p):
        if self.left(p) is not None:
            for other in self._subtree_inorder(self.left(p)):
                yield other
        yield p #子树的根节点#
        if self.right(p) is not None:
            for other in self._subtree_inorder(self.right(p)):
                yield other

In [79]:
class LinkedBinaryTree(BinaryTree):
    #二叉树的链式结构实现#
    class _Node(object):
        __slots__='_elem','_parent','_left','_right'
        def __init__(self,elem,parent=None,left=None,right=None):
            self._elem=elem
            self._parent=parent
            self._left=left
            self._right=right
    class Position(BinaryTree.Position):
        def __init__(self,container,node):
            self._container=container
            self._node=node
        
        def elem(self):
            return self._node._elem
        
        def __eq__(self,other):
            return type(self) is type(other) and self._node is other._node
        
    def __init__(self):
        self._root=None
        self._size=0
    
    def _validate(self,p):
        if not isinstance(p,self.Position):
            raise TypeError('p must be a proper Position')
        if p._container is not self:
            raise ValueError('p does not belong to this tree')
        if p._node._parent is p._node:
            raise ValueError('p is no longer valid')
        return p._node
    
    def _make_position(self,node):
        return self.Position(self,node) if node is not None else None
    
    def __len__(self):
        return self._size
    
    def root(self):
        return self._make_position(self._root)
    
    def parent(self,p):
        node=self._validate(p)
        return self._make_position(node._parent)
    
    def left(self,p):
        node=self._validate(p)
        return self._make_position(node._left)
    
    def right(self,p):
        node=self._validate(p)
        return self._make_position(node._right)
    
    def num_children(self,p):
        node=self._validate(p)
        count=0
        if node._left is not None:
            count +=1
        if node._right is not None:
            count +=1
        return count
    
    def add_root(self,e):
        if not self.is_empty():
            raise ValueError('Root exists')
        self._size+=1
        self._root=self._Node(e)
        return self._make_position(self._root)
    
    def add_left(self,p,e):
        node=self._validate(p)
        if node._left is not None:
            raise ValueError('Left child exists')
        self._size+=1
        node._left = self._Node(e)
        return self._make_position(node._left)
    
    def add_right(self,p,e):
        node=self._validate(p)
        if node._right is not None:
            raise ValueError('Right child exists')
        self._size+=1
        node._right = self._Node(e)
        return self._make_position(node._right)
    
    def replace(self,p,e):
        node=self._validate(p)
        old=node._elem
        node._elem=e
        return old
    
    def delete(self,p):
        if self.num_children(p)==2:
            raise ValueError('p has two children')
        node =self._validate(p)
        child =node._left if node._left else node._right
        
        if child is not None:
            child._parent=node._parent
        if node is self._root:
            self._root=child
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
        
        self._size -=1
        node._parent=node
        return node._elem
    
    def _attach(self,p,t1,t2):
        node=self._validate(p)
        if not self.is_leaf(p):raise ValueError('position must be leaf')
        if not type(self) is type(t1) is type(t2):
            raise TypeError('Tree type must match')
        self._size+=len(t1)+len(t2)
        if not t1.is_empty():
            t1._root._parent=node
            node._left=t1._root
            t1._root=None
            t1._size=0
        if not t2.is_empty():
            t2._root._parent=node
            node._right=t2._root
            t2._root=None
            t2._size=0
        

In [80]:
T=LinkedBinaryTree()#初始化#
a=T.add_root('a')
print(a)
b=T.add_left(a,'b')
c=T.add_right(a,'c')

d=T.add_left(b,'d')
e=T.add_right(b,'e')

f=T.add_left(e,'f')
g=T.add_right(e,'g')

results_preorder = [p.elem() for p in T._preorder()]
print(f'results_preorder:{results_preorder}')

results_postorder = [p.elem() for p in T._postorder()]
print(f'results_postorder:{results_postorder}')

results_inorder = [p.elem() for p in T._inorder()]
print(f'results_inorder:{results_inorder}')

results_breadthfirst = [p.elem() for p in T.breadthfirst()]
print(f'results_breadthfirst:{results_breadthfirst}')

results_preorder:['a', 'b', 'd', 'e', 'f', 'g', 'c']
results_postorder:['d', 'f', 'g', 'e', 'b', 'c', 'a']
results_inorder:['d', 'b', 'f', 'e', 'g', 'a', 'c']
results_breadthfirst:['a', 'b', 'c', 'd', 'e', 'f', 'g']


In [86]:
class ExpressionTree(LinkedBinaryTree):
    #表达式树#
    def __init__(self,token,left=None,right=None):
        super().__init__()#继承了父类的所有属性，方法是自动继承的#
        if not isinstance(token,str):
            raise TypeError('token should be a string')
        self.add_root(token)
        if left is not None and right is not None:
            if token not in '+-*/':
                raise ValueError('token should be a valid operator')
            
            self._attach(self.root(),left,right)
            
    def __repr__(self):
        pieces=[]
        self._parenthesize_recur(self.root(),pieces)
        return str(pieces)
    
    def _parenthesize_recur(self,p,result):
        if self.is_leaf(p):#是叶子节点#
            result.append(p.elem())
        else:#是内部节点，递归#
            result.append('(')
            self._parenthesize_recur(self.left(p),result)#挂左孩子结果#
            result.append(p.elem())
            self._parenthesize_recur(self.right(p),result)#挂右孩子结果#
            result.append(')')

def build_expression_tree(tokens):
        S=[]
        for t in tokens:
            if t in '+-*/':
                S.append(t)
            elif t not in '()':
                S.append(ExpressionTree(t))
            elif t==')':
                right=S.pop()
                op=S.pop()
                left=S.pop()
                S.append(ExpressionTree(op,left,right))
        return S.pop()    
    
if __name__=='__main__':
    tokens='((3+2)/(5*8))'
    E=build_expression_tree(tokens)
    print(E)
    

['(', '(', '3', '+', '2', ')', '/', '(', '5', '*', '8', ')', ')']
